# 06. Common Messy Datasets

### Objectives
After this lesson you should be able to...

+ Identify the type of messy data
+ Transform any messy dataset into a tidy data set
+ Use the **`str`** accessor methods to parse strings

### Prepare for this lesson by...
+ Read the entire page [working with text data](http://pandas.pydata.org/pandas-docs/stable/text.html)

### Introduction
The previous notebooks focused on one particular type of messy dataset. A dataset where the column names are actually values and not variables. This was illustrated with the simple dataset of counts of fruits. The **`melt`** method will quickly tidy these basic datasets, but often is the case that datasets take more manipulation to make them tidy.

### Most Common Messy Data Problems

1. Column headers are values, not variable names.
1. Multiple variables are stored in one column.
1. Variables are stored in both rows and columns.
1. Multiple types of observational units are stored in the same table.
1. A single observational unit is stored in multiple tables

The first type of messy data was covered in the previous notebook. This notebook will cover the next three examples.

# Multiple variables are stored in one column
A tidy data set needs values of a single variable stored in one column.

### Column names are values in the column
Column names appear directly as values in a single column and the value of these variables are in another column. Notice below how the **`Value`** column has both numeric and string data types and the **`Info`** column isn't a variable at all but column names.

In [1]:
import pandas as pd
df = pd.DataFrame(data={'State': ['Texas', 'Arizona', 'Florida'] * 3,
                        'Info': ['Age'] * 3 + ['Salary'] * 3 + ['Hair Color'] * 3, 
                        'Value': [10, 15, 20, 3, 4, 5, 'Brown', 'Pink','Red']},
                 columns=['State', 'Info', 'Value'])
df

,State,Info,Value
0,Texas,Age,10
1,Arizona,Age,15
2,Florida,Age,20
3,Texas,Salary,3
4,Arizona,Salary,4
5,Florida,Salary,5
6,Texas,Hair Color,Brown
7,Arizona,Hair Color,Pink
8,Florida,Hair Color,Red


### The fix
This dataset is 'overly melted', so pivoting it with the **`pivot`** method will make it tidy.

In [2]:
df.pivot(index='State', columns='Info', values='Value')

Info,Age,Hair Color,Salary
State,,,
Arizona,15,Pink,4
Florida,20,Red,5
Texas,10,Brown,3


In [3]:
df_tidy = df.pivot(index='State', columns='Info', values='Value').reset_index()
df_tidy.columns.name = None
df_tidy

,State,Age,Hair Color,Salary
0,Arizona,15,Pink,4
1,Florida,20,Red,5
2,Texas,10,Brown,3


## Checking data types
Whenever we have a mix of variables in a single column, you might also have a mix of data types. It's important to check the data types after reshaping the data.

In [4]:
df_tidy.dtypes

State         object
Age           object
Hair Color    object
Salary        object
dtype: object

## Changing data types
Both **`Age`** and **`Salary`** should be integers but instead are objects. We need to change their data types. We have seen this previously with the function **`pd.to_numeric`**, but you will also see the **`astype`** method used. They both do nearly the same thing, except **`pd.numeric`** gives you more options (which were needed in a previous notebook). We will use each here.

In [5]:
df_tidy['Age'] = df_tidy['Age'].astype('int')
df_tidy['Salary'] = pd.to_numeric(df_tidy['Salary'])

In [6]:
df_tidy.dtypes

State         object
Age            int64
Hair Color    object
Salary         int64
dtype: object

# Two or more values are stored in the same cell
Two or more values of the same variable or different variable can be stored in the same cell in a DataFrame. You will need to extract the desired quantities. You will need to parse the strings to extract the relevant variables. Let's take a look at a dataset with multiple variables stored in a single cell.

In [7]:
geo = pd.DataFrame({'City':['Houston', 'Dallas', 'Austin'], 
                   'Geolocation':['(29.7604° N, 95.3698° W)', 
                                  '32.7767° N, 96.7970° W', 
                                  '30.2672° N, 97.7431° W']})
geo

,City,Geolocation
0,Houston,"(29.7604° N, 95.3698° W)"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


### Identify the Variables
The first step in tidying data is identifying the variables. The **`Geolocation`** column has quite a lot of information packed into it. We will parse it into 4 separate variables.
+ latitude 
+ latitude direction
+ longitude
+ longitude direction

### Extracting information with regular expressions using the `str` accessor
The **`extract`** string method takes a regular expression with **capture groups** and returns each captured group as a new column.

Our regular expression has 4 capture groups. One for each variable. 

```
([0-9.]+)[^A-Z]+([A-Z])\D+([0-9.]+)[^A-Z]+([A-Z])
```

### Explanation
[Regex101][1] is a fantastic site to practice your regular expressions. It gives you detailed explanations of what your regular expression is doing in plain English. You can even save regular expressions as direct links. The above is regex is [saved here][2]

### My Explanation
* `([0-9.]+)` - This is a capture group that searches for one or more of the digits 0-9 and the literal `.`
* `[^A-Z]+` - This searches for one or more non-uppercase letters
* `([A-Z])` - This is a capture group for exactly one uppercase letter
* `\D+` - Finds one or more non-digits
* The regex finishes with the first three bullet points repeating

[1]: https://regex101.com/
[2]: https://regex101.com/r/XHFHcH/4

In [8]:
geo_extract = geo['Geolocation'].str.extract(r'([0-9.]+)[^A-Z]+([A-Z])\D+([0-9.]+)[^A-Z]+([A-Z])')
geo_extract

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


### Column names
Pandas defaults the column names of the resulting DataFrame to integers. We would like these new columns appended to our original DataFrame.

### Creating multiple new column names
It is possible to create several new columns in our original DataFrame by simply assigning the above resulting DataFrame to a selection of new column names as a list.

In [9]:
geo[['latitude', 'latitude direction', 'longitude', 'longitude direction']] = geo_extract
geo

,City,Geolocation,latitude,latitude direction,longitude,longitude direction
0,Houston,"(29.7604° N, 95.3698° W)",29.7604,N,95.3698,W
1,Dallas,"32.7767° N, 96.7970° W",32.7767,N,96.7970,W
2,Austin,"30.2672° N, 97.7431° W",30.2672,N,97.7431,W


### Dropping the original column
We can remove the **`Geolocation`** column as it is not relevant.

In [10]:
geo = geo.drop(columns='Geolocation')
geo

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


## Check and change Data Types
Both latitude and longitude are clearly supposed to be numeric (floats) but since they were extracted from a string, remain as strings. Let's change them to float.

In [11]:
geo.dtypes

City                   object
latitude               object
latitude direction     object
longitude              object
longitude direction    object
dtype: object

In [12]:
geo['latitude'] = geo['latitude'].astype('float')
geo['longitude'] = geo['longitude'].astype('float')
geo.dtypes

City                    object
latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [13]:
geo

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


# Variables are stored in both rows and columns
A more difficult situation occurs when variables are stored down a column and across the column names. Pivoting and melting may have to be used together to make it tidy. Let's take a look at the example below. 

In [14]:
tfp = pd.read_csv('../data/tidy/temp_flow_pressure.csv')
tfp

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


### Identifying the Variables
Identifying variables in this dataset is not as straightforward as it is in others. There are variables in a column and as column names.

We can use the following as variables:

* Group
* Pressure
* Temperature
* Flow
* Years

The years are column names and the pressure, temperature, and flow are values in the property column. The Group column is the only one in the correct place.

### Melt the years
Tidying this particular dataset must happen in multiple stages. We won't be able to tidy each variable at the same time. We will begin by melting the year column names into a single column.

In [15]:
tfp_melt = tfp.melt(id_vars=['Group', 'Property'], 
                  value_vars=['2012', '2013', '2014', '2015', '2016'],
                  var_name='Year')
tfp_melt.head(10)

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
5,B,Flow,2012,887
6,A,Pressure,2013,873
7,A,Temperature,2013,1038
8,A,Flow,2013,806
9,B,Pressure,2013,877


### Need to pivot Property
We now need to pivot the **Property** column so that the values become column names, and keep Group and Year as columns. The values will come from the **`value`** column.

### Problem! `pivot` only works with a single column as the index
If we try and pivot by passing a list of values to the **`index`** parameter, we get an error. Pandas actually thinks we are using the `['Group', 'Year']` not as column names but as values to pivot.

In [16]:
tfp_melt.pivot(index=['Group', 'Year'], columns='Property', values='value')

ValueError: Length of passed values is 30, index implies 2

### Must use `pivot_table`
The **`pivot_table`** method does allow us to keep multiple columns in the index.

In [17]:
tfp_tidy = tfp_melt.pivot_table(index=['Group', 'Year'], columns='Property', values='value')
tfp_tidy

Property    Flow  Pressure  Temperature
Group Year                             
A     2012   819       928         1026
      2013   806       873         1038
      2014   861       814         1009
      2015   882       973         1036
      2016   856       870         1042
B     2012   887       817         1008
      2013   899       877         1041
      2014   837       914         1009
      2015   824       806         1002
      2016   873       942         1013

### Doesn't `pivot_table` aggregate the values?
Yes, **`pivot_table`** always aggregates the values in the intersection of index and columns, and by default takes the mean. In this example, there is only one value per intersection, so the mean will be exactly that value. Let's verify that there is one value per intersection.

In [18]:
tfp_melt.pivot_table(index=['Group', 'Year'], columns='Property', values='value', aggfunc='size')

Property    Flow  Pressure  Temperature
Group Year                             
A     2012     1         1            1
      2013     1         1            1
      2014     1         1            1
      2015     1         1            1
      2016     1         1            1
B     2012     1         1            1
      2013     1         1            1
      2014     1         1            1
      2015     1         1            1
      2016     1         1            1

### Clean-up

In [19]:
tfp_tidy = tfp_tidy.reset_index()
tfp_tidy.columns.name = None
tfp_tidy.dtypes

Group          object
Year           object
Flow            int64
Pressure        int64
Temperature     int64
dtype: object

In [20]:
tfp_tidy['Year'] = tfp_tidy['Year'].astype('int')
tfp_tidy.dtypes

Group          object
Year            int64
Flow            int64
Pressure        int64
Temperature     int64
dtype: object

In [21]:
tfp_tidy

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


# Steps to produce tidy data
Though, there won't be an exact set of procedures that will result in a tidy dataset, this loose guideline may help you turn messy data into tidy data.

1. Identify each variable
1. Look for variable names masquerading as column names
1. Look for column names masquerading as variable values
1. Examine the 5 types of common messy data sets to see which one your dataset most closely resembles
1. You will likely need to use **`melt`**, **`pivot`**, and **`pivot_table`**
1. You might need to separate different variables into their own DataFrame to make for easier tidying
1. Parse string data with the **`str`** accessor with the help of regular expressions.

# Exercises

### Problem 1
<span  style="color:green; font-size:16px">Make the **`tidy/country_hour_price.csv`** dataset tidy by putting all the hour columns into a single column.</span>

In [ ]:
# your code here

### Problem 2
<span  style="color:green; font-size:16px">If the resulting DataFrame from problem 1 has the strings 'HOUR1' and 'HOUR2' as values in the hour column, then extract just the numerical part of the strings and reassign the result to the hour column.</span>

In [ ]:
# your code here

### Problem 3
<span  style="color:green; font-size:16px">Read in the titanic dataset (**`tidy/titanic.csv`**) and output the first few rows. Extract the first character of the **`Ticket`** column and save it as a new column **`ticket_first`**. Find the total number of survivors, the total number of passengers, and the percentage of those who survived **by this column**. Next find the total survival rate for the entire dataset. Does this new column help predict who survived?</span>

In [ ]:
# your code here

### Problem 4
<span  style="color:green; font-size:16px">If you did problem 2 correctly, you should see that only 7% of the people with tickets that began with 'A' survived. Find the survival rate for all those 'A' tickets by **`Sex`**.</span>

In [ ]:
# your code here

### Problem 5
<span  style="color:green; font-size:16px">Find the survival rate by the last letter of the ticket. Is there any predictive power here?</span>

In [ ]:
# your code here

### Problem 6
<span  style="color:green; font-size:16px">Find the length of each passengers name and assign to the **`name_len`** column. What is the minimum and maximum name length?</span>

In [ ]:
# your code here

### Problem 7
<span  style="color:green; font-size:16px">Pass the **`name_len`** column to the **`pd.cut`** function. Also, pass a list of equal-sized cut points to the **`bins`** parameter. Assign the resulting Series to the **`name_len_cat`** column. Find the frequency count of each bin in this column.</span>

In [ ]:
# your code here

### Problem 8
<span  style="color:green; font-size:16px">Is name length a good predictor of survival?<span>

In [ ]:
# your code here

### Problem 9
<span  style="color:green; font-size:16px">Why do you think people with longer names had a better chance at survival?</span>

In [ ]:
# your code here

### Problem 10
<span  style="color:green; font-size:16px">Using the titanic dataset, do your best to extract the title from a person's name. Examples of title are 'Mr.', 'Dr.', 'Miss', etc... Save this to a column called **`title`**. Find the frequency count of the titles.</span>

In [ ]:
# your code here

### Problem 11
<span  style="color:green; font-size:16px">Does the title have good predictive value of survival?</span>

In [ ]:
# your code here

### Problem 12
<span  style="color:green; font-size:16px">Create a pivot table of survival by title and sex. Use two aggregation functions, mean and size</span>

In [ ]:
# your code here

### Problem 13
<span  style="color:green; font-size:16px">Attempt to extract the first name of each passenger into the column **`first_name`**. Are there are males and females with the same first name?</span>

In [ ]:
# your code here

### Problem 14
<span  style="color:green; font-size:16px">The past several problems have been an exercise in **feature engineering**. Several new features (columns) have been created from existing columns. Come up with your own feature and test it out on survival.</span>

In [ ]:
# your code here